In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy import interpolate
from matplotlib.pyplot import imshow

import pydicom as pyd

%matplotlib inline


## 2. Преобразование DICOM  в PNG

In [9]:
# from grayscale.py


class GrayscaleImage(object):
    
    def __init__(self, image, width, height):
        self.image = image
        self.width = width
        self.height = height
    
    def __str__(self):
      return '[%dx%d]' % (self.width, self.height)

In [10]:
# from contrast.py
import numpy as np

def shade_at_percentile(hist, percentile):
    """ Assumes the argument percentile,
        to be less 1.
    """
    n = np.sum(hist)
    cumulative_sum = np.cumsum(hist)
    
    return np.argmax(cumulative_sum/n >= percentile)

def auto_contrast(image):
    """ Apply auto contrast to an image using
        https://stackoverflow.com/questions/9744255/instagram-lux-effect/9761841#9761841
    """
    hist, _ = np.histogram(image.image.ravel(), bins=np.arange(0, 256))
    p01 = shade_at_percentile(hist, .01)
    p99 = shade_at_percentile(hist, .99)
    a = 255.0/(p99 + p01)
    b = -1.0 * a * p01

    result = (image.image.astype(float) * a) + b
    result = result.clip(0, 255.0)
    
    return GrayscaleImage(np.uint8(result), image.width, image.height)

In [11]:
# from init.py
import os
#import png
import pydicom
import numpy as np


def mri_to_png(mri_file, png_file, do_auto_contrast=False):
    """ Function to convert from a DICOM image to png
        @param mri_file: An opened file like object to read te dicom data
        @param png_file: An opened file like object to write the png data
    """

    image_2d = extract_grayscale_image(mri_file)

    if do_auto_contrast:
        image_2d = auto_contrast(image_2d)

    # Writing the PNG file
    w = png.Writer(image_2d.width, image_2d.height, greyscale=True)
    w.write(png_file, image_2d.image)

def extract_grayscale_image(mri_file):
    # Extracting data from the mri file
    plan = pydicom.read_file(mri_file)
    shape = plan.pixel_array.shape

    #Convert to float to avoid overflow or underflow losses.
    image_2d = plan.pixel_array.astype(float)

    # Rescaling grey scale between 0-255
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0
    
    #Convert to uint
    image_2d_scaled = np.uint8(image_2d_scaled)

    return GrayscaleImage(image_2d_scaled, shape[1], shape[0])


def convert_file(mri_file_path, png_file_path, auto_contrast=False):
    """ Function to convert an MRI binary file to a
        PNG image file.
        @param mri_file_path: Full path to the mri file
        @param png_file_path: Fill path to the png file
    """

    # Making sure that the mri file exists
    if not os.path.exists(mri_file_path):
        raise Exception('Source file "%s" does not exists' % mri_file_path)

    # Making sure the png file does not exist
    if os.path.exists(png_file_path):
        print('Removing existing output file %s' % png_file_path)
        os.remove(png_file_path)

    mri_file = open(mri_file_path, 'rb')
    png_file = open(png_file_path, 'wb')

    mri_to_png(mri_file, png_file, auto_contrast)

    png_file.close()




In [5]:
import png

In [6]:
def convert_folder(dcm_folder, png_folder, auto_contrast=False):
    os.makedirs(png_folder)
    for dcm_sub_folder, subdirs, files in os.walk(dcm_folder):
        for dcm_file in os.listdir(dcm_sub_folder):
            dcm_file_path = os.path.join(dcm_sub_folder, dcm_file)

            if os.path.isfile(dcm_file_path):

                rel_path = os.path.relpath(dcm_sub_folder, dcm_folder)
                png_folder_path = os.path.join(png_folder, rel_path)
                if not os.path.exists(png_folder_path):
                    os.makedirs(png_folder_path)
                png_file_path = os.path.join(png_folder_path, '%s.png' % dcm_file)

                try:
                    # Convert the actual file
                    convert_file(dcm_file_path, png_file_path, auto_contrast)
                    print('SUCCESS: %s --> %s' % (dcm_file_path, png_file_path))
                except Exception as e:
                    print('FAIL: %s --> %s : %s' % (dcm_file_path, png_file_path, e))

In [7]:
dir1 = '/media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/'
dir2 = '/media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/'

In [13]:
convert_folder(dir1,dir2,auto_contrast=False)

SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/16224011.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./16224011.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/16224012.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./16224012.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/16224021.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./16224021.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/16224022.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./16224022.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/16224031.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./16224031.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/16224032.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./16224032.dcm.png
SUCCESS: /media/integr

SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/162240t1.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./162240t1.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/162240u1.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./162240u1.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/162240v1.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./162240v1.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/162240w1.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./162240w1.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/162240x1.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./162240x1.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/162240y1.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./162240y1.dcm.png
SUCCESS: /media/integr

SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/162241w3.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./162241w3.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/162241x1.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./162241x1.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/162241x2.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./162241x2.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/162241y1.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./162241y1.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/162241y2.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./162241y2.dcm.png
SUCCESS: /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224/162241y3.dcm --> /media/integral-m/TOSHIBA EXT/dataset_fluro_mask/2016224_png/./162241y3.dcm.png
SUCCESS: /media/integr